Run The Below Code first on your colab

In [ ]:
# [Colab Cell]
!git clone https://github.com/sirano1004/e2e-clinical-llmops.git
%cd e2e-clinical-llmops
!uv pip install --system -r pyproject.toml
!pip install pyngrok

Restart Session Then run Below

In [ ]:
%cd e2e-clinical-llmops

import subprocess
import time
import os
import sys
from pyngrok import ngrok
from dotenv import load_dotenv

# 1. Load .env file if it exists (silently ignored if missing)
load_dotenv("backend/.env")

# 2. Ngrok setup (recommended to provide your own token)
#    Read token from environment variable
token = os.getenv("NGROK_AUTH_TOKEN")

# 🔥 [Core] Inject the token into ngrok configuration
if token:
    ngrok.set_auth_token(token)
    print("✅ Ngrok Auth Token set successfully!")
else:
    print("⚠️ Warning: NGROK_AUTH_TOKEN not found.")

# 3. Prepare log files
#    Writing logs to files makes debugging much easier
os.makedirs("logs", exist_ok=True)
api_log = open("logs/api.log", "w")
worker_log = open("logs/worker.log", "w")

print("🚀 Starting services...")

# 4. Start Celery worker
celery_process = subprocess.Popen(
    [
        "celery",
        "-A", "backend.worker",
        "worker",
        "--loglevel=info",
        "--pool=threads",
        "--concurrency=2"
    ],
    stdout=worker_log,
    stderr=worker_log,
    cwd=os.getcwd()  # Project root
)
print(f"✅ Celery worker started (PID: {celery_process.pid})")

# 5. Start FastAPI server (Uvicorn)
#    Update 'backend.app:app' if your entry point changes
api_process = subprocess.Popen(
    ["uvicorn", "backend.app:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=api_log,
    stderr=api_log,
    cwd=os.getcwd()
)
print(f"✅ API server started (PID: {api_process.pid})")

# 6. Open Ngrok tunnel
try:
    public_url = ngrok.connect(8000).public_url
    print(f"\n🔥 [External URL]: {public_url}\n")
    print("👉 Update your client-side API_URL with the above address")
    print("👉 To view logs: open logs/api.log from the file browser")

    # Keep the process alive and monitor child processes
    while True:
        time.sleep(10)

        if api_process.poll() is not None:
            print("❌ API server terminated!")
            break

        if celery_process.poll() is not None:
            print("❌ Celery worker terminated! Check logs/worker.log")
            break

except KeyboardInterrupt:
    print("🛑 Shutting down services...")
    api_process.terminate()
    celery_process.terminate()

    api_log.close()
    worker_log.close()
    ngrok.kill()